In [3]:
import pandas as pd
import json
import requests
from urllib.parse import quote
%matplotlib inline 

# Get crime data 
soql = "https://data.cityofchicago.org/resource/6zsd-86xi.json?$query="
soql += quote("SELECT community_area AS Community, count(*)/5 AS Crime_Rate, primary_type AS Primary_Type ")
soql += quote("WHERE 2007 < year AND year < 2013 ")
soql += quote("GROUP BY Community, Primary_Type LIMIT 10000")

resp = requests.get(soql).json()

crime_type = pd.DataFrame(resp)

# Get rid of the redundant decimal places 
crime_type.Crime_Rate = crime_type.Crime_Rate.astype(float)
crime_type.round(1) 

#ATTENTION HERE: problem with NaN remains unsolved!!!!
#crime_type.dropna(axis = 0, how = "any") 
#crime_type.tail() 

# Get socioeconomics data 
socioeconomics = pd.read_csv("Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012.csv", index_col = ['Community Area Number'])


# Merge crime data on socioeconomics data using Communty Area Number 
crime_type.rename(columns={'Community':'Community Area Number'}, inplace = True)
crime_type.set_index('Community Area Number')

mix_type = pd.DataFrame.join(crime_type, socioeconomics)
mix_type 



,Community Area Number,Crime_Rate,Primary_Type,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,0,0.2,ASSAULT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1.2,BATTERY,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939.0,39.0
2,0,0.2,CRIMINAL DAMAGE,West Ridge,7.8,17.2,8.8,20.8,38.5,23040.0,46.0
3,0,0.4,CRIMINAL TRESPASS,Uptown,3.8,24.0,8.9,11.8,22.2,35787.0,20.0
4,0,1.0,DECEPTIVE PRACTICE,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524.0,17.0
5,0,0.2,MOTOR VEHICLE THEFT,North Center,0.3,7.5,5.2,4.5,26.2,57123.0,6.0
6,0,0.4,NARCOTICS,Lake View,1.1,11.4,4.7,2.6,17.0,60058.0,5.0
7,0,0.8,OTHER OFFENSE,Lincoln Park,0.8,12.3,5.1,3.6,21.5,71551.0,2.0
8,0,1.6,THEFT,Near North Side,1.9,12.9,7.0,2.5,22.6,88669.0,1.0
9,1,4.6,ARSON,Edison Park,1.1,3.3,6.5,7.4,35.3,40959.0,8.0
